# Density Gradients

- This work has been developed for my master's dissertation: *Real-Time Volumetric Cloud Rendering*

- Inspired in [Andrew Schneider](https://www.schneidervfx.com/)

###### Tiago Gomes | [PG47702](mailto:pg47702@alunos.uminho.pt) - University of Minho | Email: [tiagotg778@gmail.com](mailto:tiagotg778@gmail.com)

---

In [ ]:
# Needed imports
from PIL import Image
import numpy as np
import random
import math

---

## Methods

In [38]:
# Define remapping function as in Schneider (2015, 2016)
def Remap(o_value : float, o_min : float, o_max : float, n_min : float, n_max : float):
    return n_min + ((o_value - o_min) / (o_max - o_min)) * (n_max - n_min)

# Method to generate the density gradient texture
def DensityGradientGeneration(imgsize : tuple[int, int], isBottom : bool,
                               omin1: float, omax1: float, nmin1: float, nmax1: float,
                               omin2: float, omax2: float, nmin2: float, nmax2: float) -> Image:
    # RGB texture:
    generated_image = Image.new('RGB', imgsize)

    # scale factor to convert to rgb
    rgb_scale = 256

    # Generate the gradient texture
    for y in range(imgsize[1]):
        for x in range(imgsize[0]):
            # y is the axis for the density value. Got to normalize it
            norm_y = y / imgsize[1]

            distanceToPoint = math.sqrt((x - imgsize[0]*4) ** 2 + (y - imgsize[1]*4) ** 2)
            distanceToPoint = float(distanceToPoint) / (math.sqrt(2) * imgsize[0]*4)

            pixel_value = 0

            if(isBottom):
                pixel_value = rgb_scale * np.clip((Remap( 1 - (norm_y * distanceToPoint), omin1, omax1, nmin1, nmax1) 
                                                 * Remap( 1 - (norm_y * distanceToPoint), omin2, omax2, nmin2, nmax2)), 0, 1)
            else:
                pixel_value = rgb_scale * np.clip((Remap(norm_y, omin1, omax1, nmin1, nmax1) 
                                                 * Remap(norm_y, omin2, omax2, nmin2, nmax2)), 0, 1)

            # Place the pixel        
            generated_image.putpixel((x, (imgsize[1] - 1) - y), (int(pixel_value), int(pixel_value), int(pixel_value)))

    return generated_image

---

## Define the texture width and height

In [ ]:
# Define the texture size (512x512 for good definition)
imgsize = (512, 512)

---

## Auxiliary Methods

These methods were used to generate some of the different parameters of the remapping functions. The gradient textures in the PowerPoint presentations (which don't seem to be accurate and suffered compression) were used as the reference textures.

In [ ]:
# Method to compare two images, pixel by pixel. It returns the percentage of pixels that have the same value 
def CompareTextures(reference_image : Image, generated_image : Image) -> float:
    percentage_correct = 0

    # compare the textures
    for y in range(imgsize[1]):
        for x in range(imgsize[0]):
            generated_pixel = generated_image.getpixel((x, y))
            reference_pixel = reference_image.getpixel((x, y))

            if(generated_pixel[0] == reference_pixel[0] and generated_pixel[1] == reference_pixel[1] and generated_pixel[2] == reference_pixel[2]):
                percentage_correct += 1
    
    percentage_correct = percentage_correct / (imgsize[0]*imgsize[1]) * 100

    return percentage_correct

# Method that keeps generating images until a certain correctness percentage is reached. When it reaches that percentage, it saves an image and prints which values were used
def CorrectTexture(achieve_percentage : float, imgsize : tuple[int, int], isBottom : bool, reference_image : Image) -> Image:
    while(True):
        current_percentage = 0
        # All input values for the remap functions are in the range [0,1], so use random values for that
        random_omin1 = random.uniform(0, 1)
        random_omax1 = random.uniform(0, 1)
        random_nmin1 = random.uniform(0, 1)
        random_nmax1 = random.uniform(0, 1)

        random_omin2 = random.uniform(0, 1)
        random_omax2 = random.uniform(0, 1)
        random_nmin2 = random.uniform(0, 1)
        random_nmax2 = random.uniform(0, 1)

        generated_image = DensityGradientGeneration(imgsize, isBottom,
            stratus_omin1 = random_omin1, stratus_omax1 = random_omax1, stratus_nmin1 = random_nmin1, stratus_nmax1 = random_nmax1,
            stratus_omin2 = random_omin2, stratus_omax2 = random_omax2, stratus_nmin2 = random_nmin2, stratus_nmax2 = random_nmax2)

        # Get the current percentage
        current_percentage = CompareTextures(reference_image, generated_image)
        print(f'Current percentage: {current_percentage: .2f}; Values are: {random_omin1: .5f}, {random_omax1: .5f}, {random_nmin1: .5f}, {random_nmax1: .5f}, {random_omin2: .5f}, {random_omax2: .5f}, {random_nmin2: .5f}, {random_nmax2: .5f}')

        if(current_percentage >= achieve_percentage):
            generated_image.save('T_DensityGradient.tiff')
            return generated_image

In [ ]:
#reference_image = Image.open('name_of_the_texture')

# Test to check if generated images create what we want, compared to the reference image
#CorrectTexture(30.68, imgsize, reference_image)

---

## Generate the different gradient textures

The parameter values were not described in the presentations for the different gradient textures. These values are a result of either trial-and-error or by using the above methods.

In [ ]:
# stratus: 0.0175, 0.175, 0.005 , 0.9, 0.145, 0.18, 1.0, 0.0
# stratocumulus: 0.05, 0.19, 0.075, 0.925, 0.3, 0.485, 0.7, 0.0
# cumulus: 0.0075, 0.2, 0.0, 1.0, 0.3, 0.8, 1.0, 0.0
# bottom: 0.71547,  0.73567,  0.08276,  0.01587,  0.40550,  0.76010,  0.65148,  0.90273

stratus_gradient = DensityGradientGeneration(imgsize, False,
            omin1 = 0.0175, omax1 = 0.175, nmin1 = 0.005, nmax1 = 0.9,
            omin2 = 0.145, omax2 = 0.18, nmin2 = 1.0, nmax2 = 0.0)

stratocumulus_gradient = DensityGradientGeneration(imgsize, False,
            omin1 = 0.05, omax1 = 0.19, nmin1 = 0.075, nmax1 = 0.925,
            omin2 = 0.3, omax2 = 0.485, nmin2 = 0.7, nmax2 = 0.0)

cumulus_gradient = DensityGradientGeneration(imgsize, False,
            omin1 = 0.0075, omax1 = 0.2, nmin1 = 0.0, nmax1 = 1.0,
            omin2 = 0.3, omax2 = 0.8, nmin2 = 1.0, nmax2 = 0.0)

bottom_gradient = DensityGradientGeneration(imgsize, True,
            omin1 = 0.71547, omax1 = 0.73567, nmin1 = 0.08276, nmax1 = 0.01587,
            omin2 = 0.40550, omax2 = 0.76010, nmin2 = 0.65148, nmax2 = 0.90273)

# Save the texture gradients
stratus_gradient.save('T_Stratus_DensityGradient.tiff')
stratocumulus_gradient.save('T_Stratocumulus_DensityGradient.tiff')
cumulus_gradient.save('T_Cumulus_DensityGradient.tiff')
bottom_gradient.save('T_Bottom_DensityGradient.tiff')